# 3.5 Aprendizaje supervisado: Máquinas de vectores soporte (Support Vector Machines)

Profesor: Juan Ramón Rico (<juanramonrico@ua.es>)

## Resumen
---

Se presentarán los principios básicos de los algoritmos basados en SVM que se componen de dos fases: una dedicada a la transformación de los datos originales a una dimsesión normalmente superior (kernel) y la segunda orientada a buscar un hiperplano que separe lo mejor posible muestras de diferentes clases (hiperplane).

Parte de estos ejemplos están basados en [The RBF kernel in SVM: A Complete Guide](https://www.pycodemates.com/2022/10/the-rbf-kernel-in-svm-complete-guide.html) (kernel) y en [Implementing Support Vector Machine From Scratch](https://towardsdatascience.com/implementing-svm-from-scratch-784e4ad0bc6a) (hiperplane)

In [32]:
import numpy as np
import pandas as pd

# Máquinas de vectores soporte (SVM)

Recordemos que SVM inicialmente estaba pensada para una clasificación binaria y que su aplicación consta de dos pasos:

![](https://www.dlsi.ua.es/~juanra/UA/curso_verano_DL/images/SVM-example.svg)

 1.  **Transformación** de los **datos originales** (n-dimensiones) donde NO son separables linealmente, a otra dimensión normalmente superior, donde sí lo son. Para esta transformación se utilizan funciones concidas como `kernel`.
 2.  Búsqueda de un **hiperplano** lo más alejado posible de ambas clases que nos servirá para distinguir las dos clases.


## Las funciones de kernel más usadas son:
- Polinomial: $k(\vec{x_i}, \vec{x_j}) = (\vec{x_i} \cdot \vec{x_j})^d$
- Gaussian Radial Basis Function (RBF): $k(\vec{x_i}, \vec{x_j}) = \exp(-\gamma \| \vec{x_i} - \vec{x_j} \|^2)$ donde $\gamma>0$

## Las funciones de pérdida usadas para buscar el hiperplano son:
- Hard-margin: Mimizar $\|w\|$ tal que $y_i(\vec{w} \cdot \vec{x_i} - b) \geq 1$ para $i=1,\dots,n$
- Soft-margin: $[\frac{1}{n} \sum_{i=1}^{n} \max(0,1-y_i(\vec{w} \cdot \vec{x_i} - b))] + \lambda \frac{1}{2} \|\vec{w}\|^2$


In [33]:
## Datos

path = 'https://www.dlsi.ua.es/~juanra/UA/datasets'
path_tennis = f'{path}/tennis-en.csv'
path_covid19 = f'{path}/covid19-en.csv'

## Ejemplo

## Generación de datos sintécticos

- Vamos a generar una serie de puntos 2D pertenecientes a dos clases diferentes.
- Visualizaremos el aspecto de las muestras.

In [34]:
import numpy as np
import pandas as pd
from plotly.express import scatter
from sklearn.datasets import make_circles

X, y = make_circles(n_samples=500, noise=0.06, random_state=42)

data = pd.DataFrame({'X1':X[:, 0], 'X2':X[:, 1], 'y':y})
data['y'] = data['y'].astype('category')
data['size'] = 1
scatter(data, x='X1',y='X2', color='y', size='size', size_max=5, height=600, width=600)


## Transformación radial (RBF)



In [35]:
import numpy as np

def RBF(X_train, X_test=None, gamma=None):
      '''
      Función de kernel radial
      '''

      # Parámetro libre  gamma
      gamma = 1/len(X) if gamma is None else gamma

      # Conjunto de muestras para transformar, X_test, en base a X_train
      X_test = X_train if X_test is None else X_test

      # Fórmula del kernel RBF
      #
      # Calcula la diferencias entre componentes, las eleva al cuadrado y realiza la suma por muestra
      # Finalmente se obtiene una matriz K de NxN dimensiones, siendo N el número de muestras.
      K = np.exp(-gamma * np.sum((X_train - X_test[:,np.newaxis])**2, axis = -1))

      return K

### Visualización de los datos transformados

In [36]:
from sklearn.decomposition import PCA
from plotly.express import scatter_3d

X_kernel = RBF(X)
X_pca = PCA(n_components=3).fit_transform(X_kernel)
data = pd.DataFrame({'x1':X_pca[:, 0], 'x2':X_pca[:, 1], 'x3':X_pca[:, 2], 'y':y, 'size':1})
data['y'] = data['y'].astype('category')
scatter_3d(data, x='x1',y='x2', z='x3',color='y', size='size', size_max=10,height=600, width=600)

## Implementación de la clase SVM

A continuación mostramos un ejemplo de código de una clase llamada `SVM` que comprende su creación, entrenamiento (`fit`) y  predicción (`predict`).

In [48]:
import numpy as np
from sklearn.decomposition import PCA

class SVM:
  '''
  Clasificador de para dos clases
  '''

  def __init__(self, kernel=RBF, learning_rate=1e-3, lambda_param=1e-2, n_iters=1000):
    self.kernel = kernel
    # Este es el learning rate
    self.lr = learning_rate
    self.lambda_param = lambda_param
    self.n_iters = n_iters
    # Esto son los pesos
    self.w = None
    # Esto es el bias
    self.b = None
    self.X_train = None

  def fit(self, X, y, verbose=0):
    n_samples, n_features = X.shape
    self.X_train = X
    X_kernel = self.kernel(X, self.X_train)
    self.pca = PCA(n_components=n_features+1).fit(X_kernel)
    X_pca = self.pca.transform(X_kernel)
    self.n_features = n_features+1
    self.w, self.b = np.zeros(self.n_features), 0 # Iniciar pesos
    y_1_1 = np.where(y <= 0, -1, 1)               # Transformar {0,1} -> {-1,1}

    for epoch in range(self.n_iters):
      avg_dw, avg_db = 0, 0
      for idx, (x, y) in enumerate(zip(X_pca, y_1_1)):
        y_pred = np.dot(x, self.w) + self.b

        if y * y_pred >=1:
          # TODO: calcular las diferencias 'dw' y 'dw' si la predicción es CORRECTA
          dw = 2 * self.lambda_param * self.w  # Término de regularización solo
          db = 0
        else:  # Predicción incorrecta np.dot -> producto vectorial de dos vectores
          dw = 2 * self.lambda_param * self.w - np.dot(x ,y)  # Término de error y regularización
          db = -y  
        self.w -= self.lr * dw  # Actualizar pesos
        self.b -= self.lr * db  # Actualizar bias
        avg_dw += dw / len(X_pca)  # Actualizar promedio de pesos
        avg_db += db / len(X_pca)  # Actualizar promedio de bias
      
      
      if verbose>0 and epoch % 500 == 0:
        avg_dw /= len(X_pca)
        avg_db /= len(X_pca)
        print(f'iter: {epoch+1} dw: {avg_dw}  db: {avg_db}')
        print('w:',self.w)
        print('b:',self.b)

  def predict(self, X):
    X_predict = self.kernel(self.X_train,X)
    X_pca = self.pca.transform(X_predict)
    y_pred = np.dot(X_pca, self.w) + self.b
    sign_y_pred = np.sign(y_pred)
    return np.where(sign_y_pred == -1, 0, 1)

In [51]:
# Ejemplo de clasificación
from sklearn import metrics

X, y = make_circles(n_samples=500, noise=0.06, random_state=42)

model = SVM(kernel=RBF, learning_rate=1e-2, lambda_param=1e-2, n_iters=1000)
model.fit(X, y, verbose=1)
y_pred = model.predict(X)
print(f'Kernel RBF accuracy: {metrics.accuracy_score(y_pred, y):.4f}')

iter: 1 dw: [3.97636500e-07 3.14312780e-07 1.52519632e-05]  db: 0.0
w: [-0.00099409 -0.00078578 -0.03812991]
b: 3.469446951953614e-18
iter: 501 dw: [-1.54498810e-21 -1.95156391e-21  8.13151629e-22]  db: 0.0
w: [-0.01044525 -0.00825647 -0.40064363]
b: 3.469446951953614e-18
Kernel RBF accuracy: 0.9320


## Ejercicio `tenis` (SVM)

In [50]:
import pandas as pd
#from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder


'''
  Hay que cambiar las categorías de las variables por número enteros
  Usaremos una función de preprocesado de sklearn llamada LabelEncoder
'''
data = pd.read_csv(path_tennis)
le = LabelEncoder()
data["weather"] = le.fit_transform(data["weather"])
data["temperature"] = le.fit_transform(data["temperature"])
data["humidity"] = le.fit_transform(data["humidity"])
data["wind"] = le.fit_transform(data["wind"])
data["play"] = le.fit_transform(data["play"])

X = data.drop('play', axis=1)
y = data['play']


# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo

model = SVM(kernel=RBF, learning_rate=1e-2, lambda_param=1e-2, n_iters=1000)
model.fit(X.values, y.values, verbose=1)
y_pred = model.predict(X.values)
print(f'Kernel RBF accuracy: {metrics.accuracy_score(y_pred, y.values):.4f}')

iter: 1 dw: [-0.00156042  0.00844399 -0.00198727  0.00203348 -0.0025821 ]  db: -0.02040816326530612
w: [ 0.00305842 -0.01655022  0.00389505 -0.00398562  0.00506092]
b: 0.04
iter: 501 dw: [ 0.00052448  0.00254572  0.00045602  0.00036984 -0.00025515]  db: -0.00510204081632653
w: [-0.13118157 -1.91514592  1.07169981 -1.05307159  0.85803877]
b: 0.6000000000000003
Kernel RBF accuracy: 0.7857


## Ejercicio `covid19` (SVM)

In [54]:
import pandas as pd
from sklearn.svm import SVC

'''
  Hay que cambiar las categorías de las variables por número enteros
  Usaremos una función de preprocesado de sklearn llamada LabelEncoder
'''
data = pd.read_csv(path_covid19)
print(data.head())
data["Fever"] = le.fit_transform(data["Fever"])
data["Cough"] = le.fit_transform(data["Cough"])
data["Respiratory Problems"] = le.fit_transform(data["Respiratory Problems"])
data["Infected"] = le.fit_transform(data["Infected"])

# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo

X = data.drop('Infected', axis=1)
y = data['Infected']

model = SVM(kernel=RBF, learning_rate=1e-2, lambda_param=1e-2, n_iters=1000)
model.fit(X.values, y.values, verbose=1)
y_pred = model.predict(X.values)
print(f'Kernel RBF accuracy: {metrics.accuracy_score(y_pred, y.values):.4f}')

   Id Fever Cough Respiratory Problems Infected
0   1    No    No                   No       No
1   2   Yes   Yes                  Yes      Yes
2   3   Yes   Yes                   No       No
3   4   Yes    No                  Yes      Yes
4   5   Yes   Yes                  Yes      Yes
iter: 1 dw: [ 0.00643477 -0.02332265 -0.00260864  0.00238738  0.00329467]  db: -0.01020408163265306
w: [-0.01261215  0.0457124   0.00511293 -0.00467926 -0.00645755]
b: 0.019999999999999997
iter: 501 dw: [-0.00176161 -0.00150176 -0.00038864  0.00063435  0.00064686]  db: 0.00510204081632653
w: [-0.23645464  1.26748079  0.27536767 -1.09643965 -1.2547788 ]
b: 0.2
Kernel RBF accuracy: 0.6429


# SVM (scikit-learn)

Vamos a obtener el resultado de una clasificación de muestras correspondientes a dos clases, cuya entrada son dos características numéricas (2D) que formando círculos concéntricos. Usaremos el paquete `scikit-learn` para entrenar los modelos con diferentes kernels y mostraremos los resultados.

In [55]:
import numpy as np
from sklearn.datasets import make_circles
from sklearn.svm import SVC
from sklearn import metrics

# Crear los muestras de ejemplo
X, y = make_circles(n_samples=500, noise=0.06, random_state=42)

# Como parámetro usaremos diferentes tipo de kernel
for kernel in ['linear','poly','rbf']:
  model = SVC(kernel=kernel)
  model.fit(X, y)
  y_pred = model.predict(X)
  print(f'Kernel: {kernel:10s} accuracy: {metrics.accuracy_score(y_pred, y):.4f}')

Kernel: linear     accuracy: 0.4960
Kernel: poly       accuracy: 0.5660
Kernel: rbf        accuracy: 0.9320


## Ejercicio `tenis` (scikit-learn)

In [56]:
import pandas as pd
from sklearn.svm import SVC

data = pd.read_csv(path_tennis)
le = LabelEncoder()
data["weather"] = le.fit_transform(data["weather"])
data["temperature"] = le.fit_transform(data["temperature"])
data["humidity"] = le.fit_transform(data["humidity"])
data["wind"] = le.fit_transform(data["wind"])
data["play"] = le.fit_transform(data["play"])

X = data.drop('play', axis=1)
y = data['play']


# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo


# Como parámetro usaremos diferentes tipo de kernel
for kernel in ['linear','poly','rbf']:
  model = SVC(kernel=kernel)
  model.fit(X, y)
  y_pred = model.predict(X)
  print(f'Kernel: {kernel:10s} accuracy: {metrics.accuracy_score(y_pred, y)}')

Kernel: linear     accuracy: 0.8571428571428571
Kernel: poly       accuracy: 0.9285714285714286
Kernel: rbf        accuracy: 0.7857142857142857


## Ejercicio `covid19` (scikit-learn)

In [57]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder

'''
  Hay que cambiar las categorías de las variables por número enteros
  Usaremos una función de preprocesado de sklearn llamada LabelEncoder
'''
data = pd.read_csv(path_covid19, index_col=0)

# TODO: Adaptar los datos para la entrada (X) y salida (y) del algoritmo
print(data.head())
data["Fever"] = le.fit_transform(data["Fever"])
data["Cough"] = le.fit_transform(data["Cough"])
data["Respiratory Problems"] = le.fit_transform(data["Respiratory Problems"])
data["Infected"] = le.fit_transform(data["Infected"])



# Como parámetro usaremos diferentes tipo de kernel
for kernel in ['linear','poly','rbf']:
  model = SVC(kernel=kernel)
  model.fit(X, y)
  y_pred = model.predict(X)
  print(f'Kernel: {kernel:10s} accuracy: {metrics.accuracy_score(y_pred, y):.4f}')

   Fever Cough Respiratory Problems Infected
Id                                          
1     No    No                   No       No
2    Yes   Yes                  Yes      Yes
3    Yes   Yes                   No       No
4    Yes    No                  Yes      Yes
5    Yes   Yes                  Yes      Yes
Kernel: linear     accuracy: 0.8571
Kernel: poly       accuracy: 0.9286
Kernel: rbf        accuracy: 0.7857
